In [76]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 데이터마트용 db 와 계정 생성

- DB : etlmysqlDM
- 계정 : bigDM@'%', 'bigDMl1234@'   

In [77]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 

In [78]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='D2Coding')

In [79]:
# 기준일 생성 함수
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

### Data Mart 의 data 이동
- dw db(etlmysql) -> 처리프로그램 -> dm db(etlmysqlDM)

## DB 연결 정보
- conf_dw : 데이터 추출
- conf_dm : 데이터 저장(특별한 경우 dm db에서 data를 추출할 수도 있다)

### JDBC

In [5]:
conf_dw = {
      'url':'jdbc:mysql://localhost:3306/etlmysql?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigMysql',
      'password':'bigMysql1234@'   
      }
}

conf_dm = {
      'url':'jdbc:mysql://localhost:3306/etlmysqlDM?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigDM',
      'password':'bigDMl1234@'   
      }
}


In [6]:
# table 저장/ 반환 관련된 함수를 구성함
# config : db 연결정보
# spark.read.jdbc(): db에서 명시된 테이블을 반환받는 모듈
def find_data(config, table_name):
    return spark.read.jdbc(url=config['url'], table = table_name, properties = config['props'])

def save_data(config, df, table_name):
    return df.write.jdbc(url=config['url'], table = table_name, mode = 'append', properties = config['props'])

## Load DataMart

### 1. 지역별 단위면적(km)당 인구수와 코로나 확진자 수 데이터 저장

- base data load
    - 지역별 인구와 면적 data : dw db(etlmysql)에 LOC table 로 저장되어 있다.
    - 코로나 확진자 data : dw db(etlmysql)에 CORONA_PATIENTS table 로 저장되어 있다.

In [7]:
popu = find_data(conf_dw, 'LOC')
patients = find_data(conf_dw, 'CORONA_PATIENTS') 

In [8]:
patients.columns
type(popu) # DF 형태
popu.columns

['LOC', 'DEATH_CNT', 'DEF_CNT', 'LOC_OCC_CNT', 'QUR_RATE', 'STD_DAY']

pyspark.sql.dataframe.DataFrame

['LOC', 'AREA', 'POPULATION']

- dw db에 저장되는 data는 자주 갱신이 가능
- 정규화가 필요
    - df 생성시 db table 정규화 감안해서 생성해야 한다.
    - 기본키에 대한 컬럼 생성이 필요
    - 기본키
        - 컬럼 내에서 값이 유니크해야함
        - null 값 허용하지 않는다.
        - 기본키는 증가 가능한 고유 id 를 주로 사용한다.
    - 기본키(고유id) 생성하기 위한 모듈을 사용
        - spark 데이터 프레임의 StatFunction 패키지 함수 중 monotonically_increasing_id 사용

In [9]:
['LOC', 'DEATH_CNT', 'DEF_CNT', 'LOC_OCC_CNT', 'QUR_RATE', 'STD_DAY']
# 지역   누적사망자수  누적 발생수  지역발생수     만명당 발생률  기준일자

['LOC', 'DEATH_CNT', 'DEF_CNT', 'LOC_OCC_CNT', 'QUR_RATE', 'STD_DAY']

In [10]:
## 면적과 인구 data
popu.count()
patients.count()

17

544

In [11]:
# 면적 인구 데이터와 코로나 현황 데이터를 결합(join)
popu.join(patients, on = 'LOC').tail(20) # df의 마지막 20행을 row 객체 형태로 추출하기

[Row(LOC='충북', AREA='7407 ', POPULATION='1633472 ', DEATH_CNT='921', DEF_CNT='855269', LOC_OCC_CNT='1685', QUR_RATE='53540', STD_DAY='2022-12-02'),
 Row(LOC='충북', AREA='7407 ', POPULATION='1633472 ', DEATH_CNT='966', DEF_CNT='908476', LOC_OCC_CNT='2372', QUR_RATE='56871', STD_DAY='2022-12-29'),
 Row(LOC='충북', AREA='7407 ', POPULATION='1633472 ', DEATH_CNT='942', DEF_CNT='884139', LOC_OCC_CNT='2177', QUR_RATE='55348', STD_DAY='2022-12-17'),
 Row(LOC='충북', AREA='7407 ', POPULATION='1633472 ', DEATH_CNT='962', DEF_CNT='900403', LOC_OCC_CNT='1827', QUR_RATE='56366', STD_DAY='2022-12-25'),
 Row(LOC='충북', AREA='7407 ', POPULATION='1633472 ', DEATH_CNT='938', DEF_CNT='879966', LOC_OCC_CNT='2078', QUR_RATE='55086', STD_DAY='2022-12-15'),
 Row(LOC='충북', AREA='7407 ', POPULATION='1633472 ', DEATH_CNT='919', DEF_CNT='853584', LOC_OCC_CNT='1819', QUR_RATE='53435', STD_DAY='2022-12-01'),
 Row(LOC='충북', AREA='7407 ', POPULATION='1633472 ', DEATH_CNT='945', DEF_CNT='885942', LOC_OCC_CNT='1803', QUR_R

In [12]:
## 면적대비 인구수와 코로나 현황
pop_patients = popu.join(patients, on = 'LOC')\
                    .select('LOC',
                           ceil(col('POPULATION')/col('AREA')).alias('popu_density'),
                            'QUR_RATE', 'STD_DAY')\
                    .orderBy('STD_DAY')
pop_patients.show()

+----+------------+--------+----------+
| LOC|popu_density|QUR_RATE|   STD_DAY|
+----+------------+--------+----------+
|경남|         321|   47784|2022-11-21|
|대전|        2722|   53355|2022-11-21|
|광주|        2920|   53758|2022-11-21|
|강원|          93|   51433|2022-11-21|
|부산|        4411|   46594|2022-11-21|
|경기|        1366|   52912|2022-11-21|
|울산|        1072|   50498|2022-11-21|
|인천|        2831|   52164|2022-11-21|
|전남|         152|   47423|2022-11-21|
|전북|         226|   50078|2022-11-21|
|제주|         378|   51069|2022-11-21|
|충남|         265|   50511|2022-11-21|
|충북|         221|   52303|2022-11-21|
|서울|       16093|   54630|2022-11-21|
|대구|        2733|   48691|2022-11-21|
|세종|         811|   56400|2022-11-21|
|경북|         141|   46329|2022-11-21|
|부산|        4411|   47583|2022-12-01|
|서울|       16093|   55820|2022-12-01|
|전남|         152|   48291|2022-12-01|
+----+------------+--------+----------+
only showing top 20 rows



In [13]:
pop_patients.head(5)
pop_patients.tail(5)

[Row(LOC='광주', popu_density=2920, QUR_RATE='53758', STD_DAY='2022-11-21'),
 Row(LOC='서울', popu_density=16093, QUR_RATE='54630', STD_DAY='2022-11-21'),
 Row(LOC='대구', popu_density=2733, QUR_RATE='48691', STD_DAY='2022-11-21'),
 Row(LOC='경기', popu_density=1366, QUR_RATE='52912', STD_DAY='2022-11-21'),
 Row(LOC='대전', popu_density=2722, QUR_RATE='53355', STD_DAY='2022-11-21')]

[Row(LOC='전남', popu_density=152, QUR_RATE='51698', STD_DAY='2022-12-31'),
 Row(LOC='전북', popu_density=226, QUR_RATE='54446', STD_DAY='2022-12-31'),
 Row(LOC='제주', popu_density=378, QUR_RATE='54075', STD_DAY='2022-12-31'),
 Row(LOC='충남', popu_density=265, QUR_RATE='55389', STD_DAY='2022-12-31'),
 Row(LOC='충북', popu_density=221, QUR_RATE='57114', STD_DAY='2022-12-31')]

In [14]:
# 각 행별로 고유 인덱스 컬럼 추가
pop_patients = pop_patients .withColumn('CP_IDX', monotonically_increasing_id())
pop_patients.show()

+----+------------+--------+----------+------+
| LOC|popu_density|QUR_RATE|   STD_DAY|CP_IDX|
+----+------------+--------+----------+------+
|강원|          93|   51433|2022-11-21|     0|
|경기|        1366|   52912|2022-11-21|     1|
|경남|         321|   47784|2022-11-21|     2|
|경북|         141|   46329|2022-11-21|     3|
|광주|        2920|   53758|2022-11-21|     4|
|대구|        2733|   48691|2022-11-21|     5|
|대전|        2722|   53355|2022-11-21|     6|
|부산|        4411|   46594|2022-11-21|     7|
|서울|       16093|   54630|2022-11-21|     8|
|세종|         811|   56400|2022-11-21|     9|
|울산|        1072|   50498|2022-11-21|    10|
|인천|        2831|   52164|2022-11-21|    11|
|전남|         152|   47423|2022-11-21|    12|
|전북|         226|   50078|2022-11-21|    13|
|제주|         378|   51069|2022-11-21|    14|
|충남|         265|   50511|2022-11-21|    15|
|충북|         221|   52303|2022-11-21|    16|
|강원|          93|   52575|2022-12-01|    17|
|경기|        1366|   54097|2022-12-01|    18|
|경남|

In [15]:
# DM DB에 저장 - 테이블에 레코드가 insert 문
save_data(conf_dm, pop_patients,'CO_POPU_DENSITY')

Py4JJavaError: An error occurred while calling o65.jdbc.
: java.sql.SQLException: Access denied for user 'bigDM'@'localhost' (using password: YES)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:129)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:828)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:448)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:241)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:198)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProvider$.create(ConnectionProvider.scala:77)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$createConnectionFactory$1(JdbcUtils.scala:64

### 2.  백신접종 완료자와 코로나 확진자 수 데이터 저장
- 인구 10만명 당 접종 완료비율
    - 3차 접종 완료자를 기준으로

In [17]:
# 백신 data 추출
vaccine = find_data(conf_dw, 'CORONA_VACCINE')
vaccine.show()

+----+----------+----+---------+
| LOC|   STD_DAY|V_TH|    V_CNT|
+----+----------+----+---------+
|제주|2022-12-27|  v2| 585437.0|
|제주|2022-12-27|  v1|   591058|
|제주|2022-12-27|  v3| 439106.0|
|서울|2022-12-27|  v2|8257768.0|
|서울|2022-12-27|  v1|  8335298|
|서울|2022-12-27|  v3|6076429.0|
|부산|2022-12-27|  v2|2851362.0|
|부산|2022-12-27|  v1|  2879900|
|부산|2022-12-27|  v3|2140773.0|
|대구|2022-12-27|  v2|1995783.0|
|대구|2022-12-27|  v1|  2018604|
|대구|2022-12-27|  v3|1405163.0|
|광주|2022-12-27|  v2|1250568.0|
|광주|2022-12-27|  v1|  1261981|
|광주|2022-12-27|  v3| 968947.0|
|인천|2022-12-27|  v2|2550081.0|
|인천|2022-12-27|  v1|  2574233|
|인천|2022-12-27|  v3|1915266.0|
|대전|2022-12-27|  v2|1233376.0|
|대전|2022-12-27|  v1|  1245473|
+----+----------+----+---------+
only showing top 20 rows



In [21]:
# vaccine data로 가공 data 생성, 연산필드가 수치형이어야함 - V_CNT 열이 문자열이므로 형 변환
vaccine.printSchema()
vaccine =  vaccine\
            .withColumn('V_CNT', vaccine['V_CNT'].cast(IntegerType()))

vaccine.printSchema()

root
 |-- LOC: string (nullable = true)
 |-- STD_DAY: string (nullable = true)
 |-- V_TH: string (nullable = true)
 |-- V_CNT: string (nullable = true)

root
 |-- LOC: string (nullable = true)
 |-- STD_DAY: string (nullable = true)
 |-- V_TH: string (nullable = true)
 |-- V_CNT: integer (nullable = true)



In [22]:
### Long 형 df => wide 형 df

### pivot 사용 : pandas df 로 변환
pd_vaccine = vaccine.to_pandas_on_spark()

In [37]:
pd_vaccine.pivot_table(index = ['LOC', 'STD_DAY'], columns = 'V_TH', values = 'V_CNT')
pd_vaccine = pd_vaccine.reset_index()

,V_TH,v1,v2,v3
LOC,STD_DAY,,,
인천,2022-12-27,2574233.0,2550081.0,1915266.0
제주,2022-12-27,591058.0,585437.0,439106.0
부산,2022-12-27,2879900.0,2851362.0,2140773.0
대전,2022-12-27,1245473.0,1233376.0,909102.0
광주,2022-12-27,1261981.0,1250568.0,968947.0
울산,2022-12-27,968160.0,958920.0,720521.0
경기,2022-12-27,11837174.0,11723258.0,8707827.0
경북,2022-12-27,2293225.0,2269593.0,1727188.0
서울,2022-12-27,8335298.0,8257768.0,6076429.0


KeyError: 'Level unknown must be same as name (__none__)'

In [38]:
pd_vaccine.head()

,level_0,index,LOC,STD_DAY,V_TH,V_CNT
0,0,0,제주,2022-12-27,v2,585437
1,1,1,제주,2022-12-27,v1,591058
2,2,2,제주,2022-12-27,v3,439106
3,3,3,서울,2022-12-27,v2,8257768
4,4,4,서울,2022-12-27,v1,8335298


In [39]:
# spark df로 변환
vaccine = pd_vaccine.to_spark()
vaccine.show(3)

+-------+-----+----+----------+----+------+
|level_0|index| LOC|   STD_DAY|V_TH| V_CNT|
+-------+-----+----+----------+----+------+
|      0|    0|제주|2022-12-27|  v2|585437|
|      1|    1|제주|2022-12-27|  v1|591058|
|      2|    2|제주|2022-12-27|  v3|439106|
+-------+-----+----+----------+----+------+
only showing top 3 rows



In [35]:
# 지역별 인구 10만명당 백신 접종률 계산
# 인구 data 필요
popu = find_data(conf_dw, 'LOC')

# 코로나 현황 data 필요하다
patients = find_data(conf_dw, 'CORONA_PATIENTS')

In [36]:
# 인구 10만명당 백신 접종률 계산 - 3차 접종자를 기준으로
vac_rate = vaccine.join(popu, on = 'LOC')\
                    .select('LOC',
                           'STD_DAY',
                           ceil(col('V3')/col('POPULATION') * 100000).alias('Third_RATE'))

AnalysisException: cannot resolve 'V3' given input columns: [AREA, LOC, POPULATION, STD_DAY, V_CNT, V_TH, index, level_0];
'Project [LOC#790, STD_DAY#791, CEIL((('V3 / POPULATION#926) * 100000)) AS Third_RATE#950]
+- Project [LOC#790, level_0#788L, index#789L, STD_DAY#791, V_TH#792, V_CNT#793, AREA#925, POPULATION#926]
   +- Join Inner, (LOC#790 = LOC#924)
      :- Project [level_0#788L, index#789L, LOC#790, STD_DAY#791, V_TH#792, V_CNT#793]
      :  +- Project [__index_level_0__#801L, level_0#788L, index#789L, LOC#790, STD_DAY#791, V_TH#792, V_CNT#793, monotonically_increasing_id() AS __natural_order__#811L]
      :     +- Project [__index_level_0__#801L, level_0#788L, index#789L, LOC#790, STD_DAY#791, V_TH#792, V_CNT#793]
      :        +- Project [level_0#788L, index#789L, LOC#790, STD_DAY#791, V_TH#792, V_CNT#793, _w0#802L, _we0#803, (cast(_we0#803 as bigint) - cast(1 as bigint)) AS __index_level_0__#801L]
      :           +- Window [row_number() windowspecdefinition(_w0#802L ASC 

In [ ]:
vac_rate.show(1)

In [ ]:
# 백신 접종 데이터와 코로나 현황 데이터 결합 후 필요한 컬럼만 추출
co_vaccine =  vac_rate.join(patients, on = ['LOC', 'STD_DAY'])\
                        .select('LOC', 'STD_DAY', 'Third_RATE', 'QUR_RATE')

In [ ]:
# 기준키로 사용할 컬럼 생성
co_vaccine = co_vaccine.withColumn('CV_IDX', monotonically_increasing_id())

In [ ]:
co_vaccine.show(5)

In [ ]:
save_data(conf_dm, co_vaccine, 'CO_VACCINE_THIRD')

### 3. 다중이용시설과 코로나 확진자 수의 상관관계
- 다중 이용시설 data : dw db에 LOC_PFACILITY_CNT 테이블에 저장
    - 인구 10만명당 다중 이용 시설의 수
- 코로나 현황 data

In [40]:
facil = find_data(conf_dw, 'LOC_FACILITY_CNT') 
patients = find_data(conf_dw, 'CORONA_PATIENTS') 
popu = find_data(conf_dw, 'LOC')

In [41]:
facil.show(5)

+----+-------+
| LOC|FAC_CNT|
+----+-------+
|경북|    944|
|대전|    767|
|전북|    759|
|충북|    617|
|울산|    431|
+----+-------+
only showing top 5 rows



In [47]:
fac_popu = popu.join(facil, on = 'LOC')\
            .select('LOC', ceil(facil.FAC_CNT/popu.POPULATION*100000).alias('FAC_POPU'))
fac_popu.show(5)

+----+--------+
| LOC|FAC_POPU|
+----+--------+
|강원|      38|
|경기|      46|
|경남|      41|
|경북|      36|
|광주|      51|
+----+--------+
only showing top 5 rows



In [48]:
co_facil = patients.join(fac_popu, on = 'LOC')\
                    .select('LOC', 'FAC_POPU', 'QUR_RATE', 'STD_DAY')
# idx 컬럼추가
co_facil =  co_facil.withColumn('CF_IDX', monotonically_increasing_id())

In [50]:
co_facil.show()

+----+--------+--------+----------+------+
| LOC|FAC_POPU|QUR_RATE|   STD_DAY|CF_IDX|
+----+--------+--------+----------+------+
|경북|      36|   46329|2022-11-21|     0|
|경북|      36|   47480|2022-12-02|     1|
|경북|      36|   49862|2022-12-25|     2|
|경북|      36|   49165|2022-12-19|     3|
|경북|      36|   48426|2022-12-12|     4|
|경북|      36|   48904|2022-12-16|     5|
|경북|      36|   47578|2022-12-03|     6|
|경북|      36|   50017|2022-12-27|     7|
|경북|      36|   49010|2022-12-17|     8|
|경북|      36|   50287|2022-12-29|     9|
|경북|      36|   48377|2022-12-11|    10|
|경북|      36|   50511|2022-12-31|    11|
|경북|      36|   49544|2022-12-22|    12|
|경북|      36|   47712|2022-12-05|    13|
|경북|      36|   49913|2022-12-26|    14|
|경북|      36|   47668|2022-12-04|    15|
|경북|      36|   47955|2022-12-07|    16|
|경북|      36|   48274|2022-12-10|    17|
|경북|      36|   49113|2022-12-18|    18|
|경북|      36|   47381|2022-12-01|    19|
+----+--------+--------+----------+------+
only sho

In [ ]:
save_data(conf_dm, co_facil, 'CO_FACILITY')

### 4. 요일별 코로나 확진자 수를 구해보자
- 코로나 현황 data
    - 기준일을 요일로 변환: 요일별 글부으로 생성 신규확진자 수 합산

In [82]:
patients = find_data(conf_dw, 'CORONA_PATIENTS')

In [83]:
# 기준일을 요일로 변환 : dayofweek()
week_p = patients.withColumn('DAY_OF_WEEK', dayofweek(col('STD_DAY')))
week_p.show()

+----+---------+-------+-----------+--------+----------+-----------+
| LOC|DEATH_CNT|DEF_CNT|LOC_OCC_CNT|QUR_RATE|   STD_DAY|DAY_OF_WEEK|
+----+---------+-------+-----------+--------+----------+-----------+
|대전|      840| 796356|       1667|   54836|2022-12-03|          7|
|강원|     1187| 834387|       1857|   54234|2022-12-16|          6|
|경기|     7745|7508144|       8015|   55348|2022-12-12|          2|
|대전|      885| 848620|       1745|   58435|2022-12-31|          7|
|전남|      791| 903908|        982|   49318|2022-12-12|          2|
|광주|      743| 797569|        817|   55325|2022-12-05|          2|
|충남|     1383|1100702|       1918|   51938|2022-12-04|          1|
|광주|      759| 823273|       2104|   57108|2022-12-17|          7|
|인천|     1693|1630099|       4053|   55288|2022-12-17|          7|
|강원|     1210| 849583|       1261|   55222|2022-12-25|          1|
|인천|     1737|1669280|       5699|   56617|2022-12-27|          3|
|경기|     7906|7757826|      16439|   57188|2022-12-25|  

In [84]:
week_p = week_p.groupby(week_p.DAY_OF_WEEK).agg(sum(col('LOC_OCC_CNT')).alias('patients'))

In [85]:
week_p = week_p.withColumn('DAY_OF_WEEK', when(week_p.DAY_OF_WEEK == 1, 'MON')
                                        .when(week_p.DAY_OF_WEEK == 2, 'TUE')
                                        .when(week_p.DAY_OF_WEEK == 3, 'WED')
                                        .when(week_p.DAY_OF_WEEK == 4, 'THR')
                                        .when(week_p.DAY_OF_WEEK == 5, 'FRI')
                                        .when(week_p.DAY_OF_WEEK == 6, 'SAT')
                                        .when(week_p.DAY_OF_WEEK == 7, 'SUN')
                          )
week_p.show()

+-----------+--------+
|DAY_OF_WEEK|patients|
+-----------+--------+
|        MON|217923.0|
|        SAT|315699.0|
|        WED|339332.0|
|        FRI|339383.0|
|        THR|334716.0|
|        SUN|311539.0|
|        TUE|123733.0|
+-----------+--------+



In [86]:
pd_p = week_p.to_pandas_on_spark()
pd_p = pd_p.pivot_table(columns = 'DAY_OF_WEEK', values = 'patients')
pd_p

DAY_OF_WEEK,FRI,MON,SAT,SUN,THR,TUE,WED
patients,339383.0,217923.0,315699.0,311539.0,334716.0,123733.0,339332.0


In [88]:
pd_p = pd_p.reset_index()

In [93]:
week_p = pd_p.to_spark()
# week_p.withColumn('STD_DAY', current_date().cast('string')).show()
week_p = week_p.withColumn('STD_DAY', lit('2022-12-31'))

In [ ]:
save_data(conf_dm, week_p, 'CO_WEEKDAY')